In [ ]:
import hashlib
import secrets
import copy

class Lab_studio_main:
    def __init__(self):
        self.main={  'id': 0,
                     'annotations':[],
                     'drafts': [],
                     'predictions': [],
                     'data': {'image': '/data/local-files/?d=scripts/test_yolo_format/imgs/001_img.png'},
                     'meta': {},
                     'created_at': '2023-11-02T08:26:26.003600Z',
                     'updated_at': '2023-11-02T10:40:33.794656Z',
                     'inner_id': 1,
                     'total_annotations': 1,
                     'cancelled_annotations': 0,
                     'total_predictions': 0,
                     'comment_count': 0,
                     'unresolved_comment_count': 0,
                     'last_comment_updated_at': None,
                     'project': 30,
                     'updated_by': 1,
                     'comment_authors': []}

    def get_main(self, path_image):
        self.main["id"]=self.main["id"]+1
        self.main["data"]["image"] = path_image
        self.main["inner_id"] = self.main["inner_id"]+1

        # return self.main.copy()
        return copy.deepcopy(self.main)

class Annotation:
    def __init__ (self):
        self.anot={ "id" : 0,
        'result': [],
        "completed_by" : 1,
        "was_cancelled" : False,
        "ground_truth" : False,
        "created_at" : '2023-11-02T08:27:31.516935Z',
        "updated_at": '2023-11-02T10:40:33.779911Z',
        "draft_created_at": '2023-11-02T08:27:25.041385Z',
        "lead_time:":  1.0,
        "prediction":  {},
        "result_count":  0,
        "unique_id":  '5a63a12d-7f3e-4ef3-a84b-99c5263762c6',
        "import_id":  None,
        "last_action":  None,
        "task":  0,
        "project":  0,
        "updated_by":  1,
        "parent_prediction":  None,
        "parent_annotation":  None,
        "last_created_by":  None}

    def get_id_hash(self, n):
        return secrets.token_urlsafe(n)
        
    def hexdigest_hash(self, n):
        return hashlib.sha1(self.get_id_hash(n).encode("UTF-8")).hexdigest()[:n]
    
    def get_unique_id(self):
        return f"{self.hexdigest_hash(8)}-{self.hexdigest_hash(4)}-{self.hexdigest_hash(4)}-{self.hexdigest_hash(12)}"

    def get_annotation(self):
        self.anot["id"]=self.anot["id"]+1
        self.anot["unique_id"] = self.get_unique_id()
        self.anot["task"] = self.anot["task"] +1

        return copy.deepcopy(self.anot)


class Result:
    def __init__ (self):
        self.result={
            'original_width': 2704,
            'original_height': 1521,
            'image_rotation': 0,
            'value': {'points': [],
            'closed': True,
            'polygonlabels': ['leaf']},
            'id': '6O1Cl8I0jD',
            'from_name': 'label',
            'to_name': 'image',
            'type': 'polygonlabels',
            'origin': 'manual'}

    def get_id_hash(self, n):
        return secrets.token_urlsafe(n)[:n]
    
    def get_results(self, points, polygonlabels, resolution ):
        self.result["value"]["points"]=points
        self.result["value"]["polygonlabels"]=[polygonlabels]
        self.result["id"] = self.get_id_hash(10)
        self.result["original_height"] = resolution[0]
        self.result["original_width"] = resolution[1]
        return copy.deepcopy(self.result)


## Prepared function

In [ ]:
import cv2 as cv
import numpy as np
import torch

def mask2polygon(mask):
    mask=mask.astype("uint8")
    # mask=mask.astype("uint16")
    # mask=mask.astype(np.uint16)

    # show_NympyMask(mask)
    # print(f"mask.max() {mask.max()}")

    contours, hierarchy = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE) # im2, contours, hierarchy
    # print(f"contours len {len(contours)}")
    if len(contours)==0:
        return 0
    contours = contours[0]
    
    ### reduce amount point counturs
    peri = cv.arcLength(contours, True)
    eps = 0.006
    smother=eps*peri
    contours = cv.approxPolyDP(contours,smother, True)
    
    size = contours.shape[0]
    polygon = contours.reshape(size,2)
    
    return polygon

def polyg_relatived(polygon, resolution):
    polygon=np.float32(polygon)
    polygon[:,:1]=polygon[:,:1]/resolution[1]*100
    polygon[:,1:]=polygon[:,1:]/resolution[0]*100
    polygon=[xi.tolist() for xi in polygon]
    return polygon

def get_mask_byID(result, idx):
    # mask_np=results[0].masks.to().data[idx]
    mask_np=result.masks.to().data[idx]
    mask_np=mask_np.cpu().detach().numpy()
    return mask_np

def resize_mask(mask_np, result):
    ## Get image size to resize mask back
    orig_h,orig_w=result.orig_shape
    size_orig=(orig_w,orig_h)

    ### Resize mask to initiale image size
    mask_np = cv2.resize(mask_np, size_orig, interpolation = cv2.INTER_AREA)
    return mask_np


def first_not_zero(zero_np):
    zero_np_sumed = zero_np.sum(axis=1) 
    zero_np_sumed
    return (zero_np_sumed !=0).argmax()

def last_not_zero(zero_np):
    zero_np_sumed = zero_np.sum(axis=1) 
    zero_np_sumed
    return np.max(np.nonzero(zero_np_sumed))

def slice_non_zero(masks):
    h,w=int(masks[0].shape[0]), int(masks[0].shape[1])
    zero=torch.zeros(h, w)
    for m in masks:
        zero=zero+m
    zero_np = zero.numpy()
    
    fnz=first_not_zero(zero_np)
    lnz=last_not_zero(zero_np)
    return (fnz,lnz)


# Show bin mask

In [ ]:
import matplotlib.pyplot as plt
import torch

def sum_matrix(masks):
    zero=torch.zeros( 1520,2704)
    for m in masks:
        zero=zero+m
    zero_np = zero.numpy()
    return zero_np

def show_NympyMask(mask_np):
    plt.imshow(mask_np, interpolation='none')
    plt.show()

# result.masks.data
masks=result.masks.data
zero_np = sum_matrix(masks)

show_NympyMask(zero_np)

In [ ]:
def first_not_zero(zero_np):
    zero_np_sumed = zero_np.sum(axis=1) 
    zero_np_sumed
    return (zero_np_sumed !=0).argmax()  # fnz

def last_not_zero(zero_np):
    zero_np_sumed = zero_np.sum(axis=1) 
    zero_np_sumed
    return np.max(np.nonzero(zero_np_sumed)) # lnz

def slice_non_zero(masks):
    h,w=int(masks[0].shape[0]), int(masks[0].shape[1])
    zero=torch.zeros(h, w)
    for m in masks:
        zero=zero+m
    zero_np = zero.numpy()
    
    fnz=first_not_zero(zero_np)
    lnz=last_not_zero(zero_np)
    return (fnz,lnz)


masks = result.masks.data
fnz,lnz=slice_non_zero(masks)
print(fnz,lnz)

In [ ]:
zero_np = sum_matrix(masks)

show_NympyMask (zero_np[fnz:lnz,:])

## Pipe YoloSef2LabelStudio

In [ ]:
# weight_model = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/002_data_experiments/002_train_leaf_tomato/10_11_2023/best_leaf.pt"
weight_model = "/Users/YaVolkonskiy/Documents/Projects/tomato/scripts/003_train_leaf_tomat_det_seg/best.pt"

pth_imgs = "/Users/YaVolkonskiy/Documents/Projects/tomato/data/001_raw_data/001_video_data/002_GO_PRO_SBER_20231101_ALL/selected_imgs/"

cls_yoloseg2LabStud = YoloSeg2LabelStudio( weight_model )
dict_labelme = cls_yoloseg2LabStud.get_labels(pth_imgs)


import json

with open('project_label_studio_selected_retina_masks_FALSE.json', 'w') as f:
    json.dump(dict_labelme, f)

In [ ]:
from ultralytics import YOLO

weight_model = "/Users/YaVolkonskiy/Documents/Projects/tomato/scripts/003_train_leaf_tomat_det_seg/best.pt"

model = YOLO(weight_model)

In [ ]:
model.names

### From labling

In [9]:
path_labels = "/Users/YaVolkonskiy/Documents/001_Projects/003_tomato/001_data/005_Разметка/001_Pool_detect_leaf_Сбер/001_Сбер_разметка_24_09_24/yolo_format_Сегментация кустов томатов_calibration"

path_images ="/Users/YaVolkonskiy/Documents/001_Projects/003_tomato/001_data/005_Разметка/001_Pool_detect_leaf_Сбер/001_Сбер_разметка_24_09_24/imgs"

In [10]:
import glob 
import numpy as np
import cv2

list_labels = glob.glob(f"{path_labels}/*.txt")
list_images = glob.glob(f"{path_images}/*.jpg")

list_images

['/Users/YaVolkonskiy/Documents/001_Projects/003_tomato/001_data/005_Разметка/001_Pool_detect_leaf_Сбер/001_Сбер_разметка_24_09_24/imgs/IMG_20240409_135011.jpg',
 '/Users/YaVolkonskiy/Documents/001_Projects/003_tomato/001_data/005_Разметка/001_Pool_detect_leaf_Сбер/001_Сбер_разметка_24_09_24/imgs/IMG20240215144314.jpg']

In [28]:
def iter_items(label_path):
    list_clas = []
    list_polyg = []

    # print(f"resolution_label {resolution_label}")
    with open(label_path) as f:
        lines = f.readlines()

    for line in lines:
        polyg_class = line[0]
        data_line = line[1:]

        arr=np.array(data_line.strip().split(" ")).astype("f2")* np.asarray(100, dtype='f2')
        arr = np.reshape(arr, (int(len(arr)/2), 2 ))

        list_clas.append(int(polyg_class))
        list_polyg.append(arr)


    return list_polyg, list_clas


def get_resolution(path_img):
    h_orig,w_orig,_ = cv2.imread(path_img).shape
    resolution_label = (h_orig,w_orig)
    return resolution_label

In [ ]:
item_one = list_labels[0]

with open(item_one) as f:
    lines = f.readlines()
    

print(len(lines)) # .split("\n")

In [26]:
# for label_path, img_path in zip(list_labels, list_images):
#         list_polyg, img_class = iter_items(label_path)

In [29]:
import os

weight_model = "/Users/YaVolkonskiy/Documents/Projects/tomato/scripts/003_train_leaf_tomat_det_seg/best.pt"

file_name = os.path.basename(weight_model)
file_name

'best.pt'

In [27]:
list_polyg

[array([[ 1.334  ,  0.     ],
        [ 1.833  ,  0.4001 ],
        [ 2.8    ,  0.9004 ],
        [ 3.4    ,  1.7    ],
        [ 4.066  ,  2.95   ],
        [ 4.668  ,  3.326  ],
        [ 5.77   ,  3.45   ],
        [ 6.336  ,  3.074  ],
        [ 6.9    ,  2.725  ],
        [ 8.03   ,  2.6    ],
        [ 8.87   ,  2.424  ],
        [ 9.03   ,  1.975  ],
        [ 9.266  ,  1.825  ],
        [ 9.6    ,  1.625  ],
        [ 9.766  ,  1.375  ],
        [10.1    ,  1.101  ],
        [10.305  ,  0.9497 ],
        [10.195  ,  0.7    ],
        [10.37   ,  0.375  ],
        [10.336  ,  0.125  ],
        [10.336  ,  0.     ],
        [ 8.03   ,  0.     ],
        [ 7.598  ,  0.05002],
        [ 6.934  ,  0.10004],
        [ 6.203  ,  0.10004],
        [ 5.832  ,  0.175  ],
        [ 5.336  ,  0.125  ],
        [ 4.867  ,  0.     ]], dtype=float16),
 array([[10.37   ,  0.     ],
        [10.47   ,  0.175  ],
        [10.766  ,  0.35   ],
        ...,
        [13.97   ,  0.05002],
        [1

In [ ]:
one_line = lines[0].strip()


img_class = one_line[0]
data_line = one_line[1:]

arr=np.array(data_line.strip().split(" "))
new_array = np.reshape(arr, (int(len(arr)/2), 2 ))*100
new_array


In [ ]:
data_line

In [ ]:
a=[1,2,3,4,5,6]
a[1:]

In [ ]:
arr=np.array(a)

new_array = np.reshape(arr, (int(len(arr)/2), 2 ))
new_array

In [ ]:
import os
import cv2
from ultralytics import YOLO

class YoloSeg2LabelStudio:
    def __init__(self, path_labels, path_images):
        self.classes = ["leaf", "stems", "tomato", "death_leaf"]
        self.list_labels = glob.glob(f"{path_labels}/*.txt")
        self.list_images = glob.glob(f"{path_images}/*.jpg")

    def get_labels(self, pth_imgs):
        l=Lab_studio_main()
        an = Annotation()
        res = Result()
        list_main_part = []
        
        for label_path, path_img in zip(self.list_labels, self.list_images):
            ## Init
            main_part=l.get_main(labelst_path)
            anotation = an.get_annotation()

            ## Main part
            list_polyg, list_clas = iter_items(label_path)
            resolution_label = get_resolution(path_img)

            for polyg, anot_clas in zip(list_polyg, list_clas):
                tmp_results = res.get_results(polyg, self.classes[anot_clas], resolution_label)
                anotation["result"].append(tmp_results)
            main_part["annotations"].append(anotation)
            list_main_part.append(main_part)

        return list_main_part

                












        # for pth_img in os.listdir(pth_imgs)[:3]:
        #     labelst_path = os.path.join(self.labelst_part_path, pth_img)
        #     pth_img_ful = os.path.join(pth_imgs,pth_img)
        #     image = cv2.imread(pth_img_ful)
        #     h_orig,w_orig,_=image.shape
        #     resolution_label = (h_orig,w_orig)
        #     print(f"resolution_label {resolution_label}")

        #     # Label studio json schema
        #     main_part=l.get_main(labelst_path)
        #     # print(f"main_part[data][image] {main_part['data']['image']}")
        #     anotation = an.get_annotation()

        #     # inference model # TODO be attentive about retina_masks=True 
        #     retina_masks=True 
        #     result=self.model(image, iou=0.35, conf=0.2, retina_masks=retina_masks )[0] # retina_masks=True

        #     # There are paddind, therefore we need to crop it
        #     if retina_masks==False: 
        #         fnz,lnz =slice_non_zero(result.masks.data) # # TODO uncoment insted prevuios string
        #         # fnz,lnz =  12,371 #  slice_non_zero(result.masks.data) # TODO check
        #         val_crop=(fnz,lnz)
        #         print(f"val_crop {val_crop}")
            

        #     amount_masks=len(result.masks.data)

        #     for idx_mask, cls in zip(range(amount_masks), result.boxes.cls):
        #         cls=int(cls)
        #         # val_crop attribute => crop slice because of padding
        #         mask_np=get_mask_byID(result, idx_mask)

        #         # TODO del because of padding
        #         if retina_masks==False:   # fnz,lnz = val_crop
        #             mask_np = mask_np[fnz:lnz,:]

        #         mask_np = resize_mask(mask_np, result)
                
        #         polyg = mask2polygon(mask_np)
        #         # print(type(polyg))
        #         if not type(polyg) == np.ndarray:
        #             continue
                    
        #         polyg=polyg_relatived(polyg, resolution_label)
        #         # Fill json schema 
        #         tmp_results = res.get_results(polyg, self.classes[cls], resolution_label)
        #         anotation["result"].append(tmp_results)
        #     print(f"result.orig_shape {result.orig_shape}")
        #     main_part["annotations"].append(anotation)
        #     list_main_part.append(main_part)
        # return list_main_part

